In [1]:
import numpy as np
import pandas as pd
import re
import math
from matplotlib import pyplot as plt

In [2]:
all_dx = np.load('../sourcedata/data/HBN/phenotype/parsed/dx_list.npy')
sub_list = np.load('../sourcedata/data/HBN/phenotype/parsed/sub_list.npy')
dx_array = np.load('../sourcedata/data/HBN/phenotype/parsed/dx_onehot.npy')

In [3]:
all_dx

array(['ADHD-Combined Type', 'ADHD-Hyperactive/Impulsive Type',
       'ADHD-Inattentive Type', 'Acute Stress Disorder',
       'Adjustment Disorders', 'Agoraphobia', 'Alcohol Use Disorder',
       'Anorexia Nervosa-Binge Eating/Purging Type',
       'Anorexia Nervosa-Restricting Type', 'Autism Spectrum Disorder',
       'Avoidant/Restrictive Food Intake Disorder',
       'Binge-Eating Disorder', 'Bipolar I Disorder',
       'Bipolar II Disorder', 'Body Dysmorphic Disorder',
       'Borderline Intellectual Functioning',
       'Borderline Personality Disorder', 'Bulimia Nervosa',
       'Cannabis Use Disorder', 'Child Neglect, Suspected',
       'Child Onset Fluency Disorder (Stuttering)',
       'Conduct Disorder-Adolescent-onset type',
       'Conduct Disorder-Childhood-onset type',
       'Conduct Disorder-Unspecified onset', 'Conversion Disorder',
       'Cyclothymic Disorder',
       'Delirium due to another medical condition',
       'Depressive Disorder Due to Another Medical Co

In [10]:
dx_df = pd.DataFrame(dx_array)
dx_df.columns = all_dx

In [25]:
sub_ind=dx_df.loc[dx_df['No Diagnosis Given'] == 1].index

In [27]:
for f in sub_list[sub_ind]:
    print(f)

NDARAA075AMK
NDARAC904DMU
NDARAE012DGA
NDARAL828WXM
NDARAM277WZT
NDARAM675UR8
NDARAN385MDH
NDARAP522AFK
NDARAT100AEQ
NDARAU447JZH
NDARAV610EY3
NDARAX283MAK
NDARBA521RA8
NDARBH024NH2
NDARBK082PDD
NDARBL117AUV
NDARBN100LCD
NDARBT436PMT
NDARBU928LV0
NDARBV150TX1
NDARBW268XPY
NDARBY518PRN
NDARCD182XT1
NDARCD401HGZ
NDARCE721YB5
NDARCG808HDJ
NDARCH514JCT
NDARCJ363KLE
NDARCM050BN2
NDARCR576EV1
NDARCT472UJ7
NDARCU891YET
NDARCW963FP9
NDARCX053GU5
NDARCY178KJP
NDARDC814WW2
NDARDH086ZKK
NDARDJ092YKH
NDARDJ825GBP
NDARDL511UND
NDARDN489EXJ
NDARDR296XHN
NDARDR804MFE
NDARDT800YVF
NDARDX872VH6
NDAREA136BDX
NDARED047DTH
NDAREF389RY2
NDAREG013BLG
NDAREG590BNY
NDAREH074NG8
NDAREK549XUQ
NDAREL622FA6
NDAREM703YFD
NDARET653TAM
NDAREU551GPC
NDAREW201WD9
NDAREW430AYU
NDAREW661NZJ
NDAREW976FNL
NDAREX065KJU
NDARFB784LDG
NDARFC233RND
NDARFG851ZNZ
NDARFJ988HKR
NDARFK610GY5
NDARFL411AT1
NDARFN854EJB
NDARFP813TL5
NDARFR109LKT
NDARFW292PBD
NDARFY075REK
NDARFZ700FAH
NDARGA563XVW
NDARGB441VVD
NDARGD507TDZ
NDARGF543PM2